In [1]:
import numpy as np
import pandas as pd

In [2]:
feature_df = pd.read_excel("./result/风控联合建模标签（2026）.xlsx")
feature_df

,特征模块,特征英文名称,字段类型,特征中文名称
0,信贷,sp_fin_loan_plat_cnt_15d,INT,近15天内金融事件平台数
1,信贷,sp_fin_loan_plat_cnt_30d,INT,近30天内金融事件平台数
2,信贷,sp_fin_loan_plat_cnt_90d,INT,近90天内金融事件平台数
3,信贷,sp_fin_loan_plat_cnt_180d,INT,近180天内金融事件平台数
4,信贷,sp_fin_loan_plat_cnt_360d,INT,近360天内金融事件平台数
...,...,...,...,...
1191,快递,sp_express_event_a04_ratio_15to30d,DOUBLE,近15天内其他快递事件次数占近30天次数的比率（保留6位小数）
1192,快递,sp_express_event_a01_ratio_30to90d,DOUBLE,近30天内快递签收事件次数占近90天次数的比率（保留6位小数）
1193,快递,sp_express_event_a02_ratio_30to90d,DOUBLE,近30天内快递投诉事件次数占近90天次数的比率（保留6位小数）
1194,快递,sp_express_event_a03_ratio_30to90d,DOUBLE,近30天内快递寄件事件次数占近90天次数的比率（保留6位小数）


In [3]:
fin_loan_feature = feature_df['特征英文名称'][feature_df['特征模块'] == '信贷']
express_feature = feature_df['特征英文名称'][feature_df['特征模块'] == '快递']
len(fin_loan_feature), len(express_feature)

(1110, 86)

In [ ]:
def create_table(config, feature_df):
    feature_merge_table = config['feature_merge_table']
    fin_loan_feature = feature_df['特征英文名称'][feature_df['特征模块'] == '信贷'].tolist()
    express_feature = feature_df['特征英文名称'][feature_df['特征模块'] == '快递'].tolist()
    feature_dict = dict(zip(feature_df['特征英文名称'], feature_df['字段类型']))
    
    feature_list = []
    for feat in list(fin_loan_feature)+list(express_feature):
        feature_list.append(f"{feat}    {feature_dict[feat]}  comment ''")

    feature_str = ', \n'.join(feature_list)
    sql = f"""
    create table if not exists {feature_merge_table}(
        mobile_md5 string,
        back_date string,
        {feature_str}
    ) comment '客户测试-离线特征结果表'
    """
    return sql
config = {
    "customer_table": "sms_bd_data.customer_test_sample_id",
    "base_id_table": "sms_bd_data.sp_sms_total_user_id",
    "fin_loan_table": "sms_bd_data.customer_test_fin_loan_feature_2026011302",
    "express_table": "sms_bd_data.customer_test_express_feature_2026011301",
    "feature_merge_table": "sms_bd_data.customer_test_sp_feature_2026011301"
}

def gen_feature_merge(config, feature_df):
    customer_table = config['customer_table']
    base_id_table = config['base_id_table']
    fin_loan_table = config['fin_loan_table']
    express_table = config['express_table']
    feature_merge_table = config['feature_merge_table']

    fin_loan_feature = feature_df['特征英文名称'][feature_df['特征模块'] == '信贷']
    express_feature = feature_df['特征英文名称'][feature_df['特征模块'] == '快递']
    fin_loan_feature = ', \n'.join([f"fin_loan.{feat}" if 'dura_d_beforep' in feat else f"nvl(fin_loan.{feat}, 0) as {feat}" for feat in fin_loan_feature])
    express_feature = ', \n'.join([f"express.{feat}" if 'dura_d_beforep' in feat else f"nvl(express.{feat}, 0) as {feat}" for feat in express_feature])
    
    base_sql = f"""
    with base_cus_sample as(
        select
            base_cus.mobile_md5,
            base_cus.back_date
        from(
            SELECT 
                phone as mobile_md5,
                the_date as back_date
            FROM {customer_table}
        )cus_sample
        inner join {base_id_table} base_cus
        on cus_sample.mobile_md5 = base_cus.phone
    )
    insert overwrite table {feature_merge_table}
    select
        base_cus.mobile_md5,
        base_cus.back_date,
        {fin_loan_feature},
        {express_feature}
    from base_cus_sample base_cus
    left join {fin_loan_table} fin_loan
    on base_cus.mobile_md5 = fin_loan.mobile_md5 and base_cus.back_date = fin_loan.back_date
    left join {express_table} express
    on base_cus.mobile_md5 = express.mobile_md5 and base_cus.back_date = express.back_date
    """
    return base_sql

In [9]:
creature_table_sql = create_table(config, feature_df)
print(creature_table_sql)


    create table if not exists sms_bd_data.customer_test_sp_feature_2026011301(
        mobile_md5 string,
        back_date string,
        sp_fin_loan_plat_cnt_15d    INT  comment '', 
sp_fin_loan_plat_cnt_30d    INT  comment '', 
sp_fin_loan_plat_cnt_90d    INT  comment '', 
sp_fin_loan_plat_cnt_180d    INT  comment '', 
sp_fin_loan_plat_cnt_360d    INT  comment '', 
sp_fin_loan_event_b01_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b02_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b03_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b04_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b05_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b06_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b07_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b08_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b09_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b10_plat_cnt_15d    INT  comment '', 
sp_fin_loan_event_b01_plat_cnt_30d    INT  comment '', 
sp_fin_loa

In [10]:
config = {
    "customer_table": "sms_bd_data.customer_test_sample_id",
    "base_id_table": "sms_bd_data.sp_sms_total_user_id",
    "fin_loan_table": "sms_bd_data.customer_test_fin_loan_feature_2026011302",
    "express_table": "sms_bd_data.customer_test_express_feature_2026011301",
    "feature_merge_table": "sms_bd_data.customer_test_sp_feature_2026011301"
}

final_sql = gen_feature_merge(config, feature_df)
print(final_sql)


    with base_cus_sample as(
        select
            base_cus.mobile_md5,
            base_cus.back_date
        from(
            SELECT 
                phone as mobile_md5,
                the_date as back_date
            FROM sms_bd_data.customer_test_sample_id
        )cus_sample
        inner join sms_bd_data.sp_sms_total_user_id base_cus
        on cus_sample.mobile_md5 = base_cus.phone
    )
    insert overwrite table sms_bd_data.customer_test_sp_feature_2026011301
    select
        base_cus.mobile_md5,
        base_cus.back_date,
        nvl(fin_loan.sp_fin_loan_plat_cnt_15d, 0) as sp_fin_loan_plat_cnt_15d, 
nvl(fin_loan.sp_fin_loan_plat_cnt_30d, 0) as sp_fin_loan_plat_cnt_30d, 
nvl(fin_loan.sp_fin_loan_plat_cnt_90d, 0) as sp_fin_loan_plat_cnt_90d, 
nvl(fin_loan.sp_fin_loan_plat_cnt_180d, 0) as sp_fin_loan_plat_cnt_180d, 
nvl(fin_loan.sp_fin_loan_plat_cnt_360d, 0) as sp_fin_loan_plat_cnt_360d, 
nvl(fin_loan.sp_fin_loan_event_b01_plat_cnt_15d, 0) as sp_fin_loan_event_b01_